In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [4]:
#hide
from fastbook import *
path = untar_data(URLs.PETS)

In [5]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock), get_items=get_image_files,
                 splitter=RandomSplitter(seed=42), 
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'), 
                 item_tfms = Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls= pets.dataloaders(path/'images')

In [6]:
dls.show_batch(nrows=1, ncols=3)

In [7]:
pets1 = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                  get_items=get_image_files,
                  splitter=RandomSplitter(seed=42), 
                  get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'))
pets.summary(path/"images")

In [8]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

In [21]:
x,y = dls.one_batch()

In [22]:
y

In [23]:
preds, _ = learn.get_preds(dl=[(x,y)])


In [24]:
preds.shape

In [25]:
len(preds[0]), preds[0].sum()

In [26]:
acts = torch.randn((6,2))*2
acts

In [27]:
acts.sigmoid()

In [28]:
(acts[:,0] - acts[:,1]).sigmoid()

In [29]:
sm_acts = torch.softmax(acts, dim=1)

In [30]:
sm_acts

In [31]:
sm_acts[2,0]

In [32]:
nn.CrossEntropy(reduction='none')(acts, targ)

In [33]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [34]:
interp.most_confused(min_val=5)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
lr_min,lr_steep = learn.lr_find()

In [28]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
lr_min, lr_steep = learn.lr_find()

In [29]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [30]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2, base_lr=3e-3)

In [35]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)

In [36]:
learn.fit_one_cycle(6, lr_max = 1e-5)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))